In [1]:
from typing import Any, Type
from pydantic import BaseModel, Field
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1)

In [2]:
class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query: str) -> Any:
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
    ],
)

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare stock market symbol'}`


Cloudflare Stock Surges on Strong Earnings, Upbeat Outlook—Key Price Level to Watch. Shares in Cloudflare (NET) surged nearly 25% in premarket trading Friday after the cybersecurity and content delivery solutions provider delivered a better-than-expected quarterly report and issued fi... Cloudflare Stock Is Surging. At Cloudflare, Inc., we promise to treat your data with respect and will not share your information with any third party. You can unsubscribe to any of the investor alerts you are subscribed to by visiting the 'unsubscribe' section below. ... End of Day Stock Quote: Sales Enterprise Sales Become a Partner Contact Sales: +1 (888) 99 FLARE ... Stock analysis for Cloudflare Inc (NET:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile. A high-level overview of Cloudflare, Inc. (NET) stock. 

{'input': "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have.",
 'output': "The stock market symbol for Cloudflare is NET. Cloudflare's stock surged nearly 25% in premarket trading after delivering a better-than-expected quarterly report and issuing an upbeat outlook.\n\nTo analyze if Cloudflare is a potential good investment, we can look at key factors such as:\n\n1. Financial Performance: Review Cloudflare's revenue growth, profitability, and cash flow to assess its financial health.\n\n2. Market Position: Evaluate Cloudflare's competitive position in the cybersecurity and content delivery solutions industry.\n\n3. Growth Potential: Consider Cloudflare's growth prospects, including expansion into new markets and services.\n\n4. Industry Trends: Analyze industry trends and how they may impact Cloudflare's future performance.\n\n5. Valuation: Assess Cloudflare's current valuation compared to i